# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [3]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [4]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [5]:
# confirm that our new database was created
print(mongo.list_database_names()) 


['admin', 'config', 'local', 'uk_food']


In [18]:
# assign the uk_food database to a variable name
food_db = mongo['uk_food']

In [20]:
# review the collections in our new database
print(food_db.list_collection_names()) 

['establishments', 'new_restaurant']


In [22]:
# review a document in the establishments collection
pprint(food_db.establishments.find_one())


{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6483838654c77ea90610ed68'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [24]:
# assign the collection to a variable
establishments = food_db['establishments']
establishments

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [25]:
pprint(food_db.new_restaurant.find_one()) 

{'BusinessName': 'Penang Flavours',
 'BusinessType': 'halal',
 '_id': ObjectId('6483d86bb7e94fa684dbe012')}


In [26]:
# db = mongo['new_restaurant']

In [13]:
# print(db.list_collection_names()) 

[]


In [30]:
# Create a dictionary for the new restaurant data
new_restaurant = {'BusinessName': 'Penang Flavours',
 'BusinessType': 'halal'}

In [31]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [32]:
# Check that the new restaurant was inserted
establishments.find_one({'BusinessName':'Penang Flavours'})


{'_id': ObjectId('648552f2c4372cc466333c7e'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'halal'}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [33]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = ['BusinessTypeID', 'BusinessType']

establishments.find_one(query, fields)

{'_id': ObjectId('6483838654c77ea90610ed6a'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [36]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    new_restaurant, 
    {'$set': 
        {'BusinessTypeID': 1}
    }
)

In [37]:
# Confirm that the new restaurant was updated
establishments.find_one({'BusinessName':'Penang Flavours'})

{'_id': ObjectId('648552f2c4372cc466333c7e'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'halal', 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [38]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName': 'Dover'})


994

In [39]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})


In [40]:
# Check if any remaining documents include Dover
pprint(establishments.count_documents({'LocalAuthorityName': 'Dover'}))


0


In [41]:
# Check that other documents remain with 'find_one'
establishments.find_one({})


{'_id': ObjectId('6483838654c77ea90610f04e'), 'FHRSID': 289353, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000002468', 'BusinessName': 'St Marys COE (aided) Primary School', 'BusinessType': 'School/college/university', 'BusinessTypeID': 7845, 'AddressLine1': 'St Marys Church Of England Primary School', 'AddressLine2': 'Warren Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6QH', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2016-09-13T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.194762', 'latitude': '51.085797'}, 'RightToReply': '', 'Distance': 4591.871474174837, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extrac

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude


Use `update_many` to convert `RatingValue` to integer numbers.

In [42]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue


In [ ]:
# Check that the coordinates and rating value are now numbers
